# Day19_3: OpenAI 파인튜닝 - LLM을 내 목적에 맞게 맞춤 제작하기 (정답)

## 학습 목표

**Part 1: 기초**
1. 파인튜닝(Fine-Tuning)의 개념을 이해하고, 프롬프트 엔지니어링, RAG와의 차이점을 설명할 수 있습니다.
2. 파인튜닝이 적합한 시나리오와 부적합한 시나리오를 구별할 수 있습니다.
3. OpenAI 파인튜닝을 위한 JSONL 데이터 형식을 이해하고 올바르게 작성할 수 있습니다.
4. 고품질 학습 데이터의 특성을 이해하고, 데이터 준비 과정을 수행할 수 있습니다.
5. OpenAI Files API를 사용하여 학습 데이터를 업로드할 수 있습니다.

**Part 2: 심화**
1. OpenAI Fine-tuning API를 사용하여 파인튜닝 작업을 생성하고 모니터링할 수 있습니다.
2. 파인튜닝된 모델을 호출하고, 기본 모델과의 성능 차이를 비교 분석할 수 있습니다.
3. 하이퍼파라미터(epochs, learning_rate_multiplier)를 조정하여 학습을 최적화할 수 있습니다.
4. 실무에서 파인튜닝을 활용하는 전략과 비용 최적화 방법을 이해합니다.

---

## 실습 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import os
import json
import time
from dotenv import load_dotenv
from openai import OpenAI

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI()

print("환경 설정 완료!")

---

## 퀴즈 정답

---

### Q1. 파인튜닝 개념 이해 (기본)

**문제**: 다음 빈칸을 채우세요.

1. 파인튜닝은 모델의 내부 (______)을 업데이트하는 과정입니다.
2. 새로운 '지식'을 추가하려면 (______)가 적합하고, '행동 방식'을 바꾸려면 (______)가 적합합니다.
3. OpenAI 파인튜닝 데이터는 (______) 형식으로 준비해야 합니다.

In [ ]:
# 정답
answer1 = "가중치(weights)"
answer2_knowledge = "RAG"
answer2_behavior = "파인튜닝"
answer3 = "JSONL"

print(f"1. {answer1}")
print(f"2. 지식: {answer2_knowledge}, 행동: {answer2_behavior}")
print(f"3. {answer3}")

**풀이 설명**

- **접근 방법**: 파인튜닝의 핵심 개념을 이해하고, 다른 LLM 최적화 기법과의 차이점 파악
- **핵심 개념**:
  - 파인튜닝은 모델의 **내부 가중치(weights)**를 추가 학습을 통해 업데이트
  - RAG는 **외부 지식**을 검색하여 제공하는 방식 (모델 변경 없음)
  - 파인튜닝은 모델의 **행동 방식, 스타일, 출력 패턴**을 변경
  - OpenAI는 JSONL 형식의 데이터를 요구 (각 줄이 독립적인 JSON 객체)
- **실무 팁**: RAG와 파인튜닝은 상호 배타적이지 않으며, 함께 사용할 수 있음

---

### Q2. JSONL 형식 작성 (기본)

**문제**: 다음 정보를 JSONL 형식으로 변환하세요.

- 시스템 메시지: "당신은 친절한 고객 상담원입니다."
- 사용자 질문: "환불은 어떻게 하나요?"
- 이상적 답변: "환불은 구매일로부터 7일 이내에 가능합니다. 마이페이지에서 신청하실 수 있습니다."

In [ ]:
# 정답
import json

# JSONL 형식의 학습 데이터
training_example = {
    "messages": [
        {"role": "system", "content": "당신은 친절한 고객 상담원입니다."},
        {"role": "user", "content": "환불은 어떻게 하나요?"},
        {"role": "assistant", "content": "환불은 구매일로부터 7일 이내에 가능합니다. 마이페이지에서 신청하실 수 있습니다."}
    ]
}

# JSONL 형식으로 출력 (한 줄)
jsonl_line = json.dumps(training_example, ensure_ascii=False)

print("JSONL 형식:")
print(jsonl_line)

# 가독성을 위한 포맷팅 출력
print("\n가독성 있는 형식:")
print(json.dumps(training_example, ensure_ascii=False, indent=2))

**풀이 설명**

- **접근 방법**: JSONL 형식의 구조 이해 (messages 배열 안에 system, user, assistant 역할)
- **핵심 개념**:
  - `messages`: 대화 내역을 담는 배열
  - `role`: system(행동 지침), user(사용자 입력), assistant(이상적 답변)
  - `content`: 각 역할의 실제 내용
- **주의사항**: 
  - ensure_ascii=False로 한글 유니코드 유지
  - JSONL은 각 줄이 독립적인 JSON이므로 indent 없이 한 줄로 작성
- **실무 팁**: system 메시지는 선택사항이지만, 일관된 행동을 위해 권장

---

### Q3. 적합한 기술 선택 (기본)

**문제**: 다음 각 상황에 가장 적합한 기술(프롬프트 엔지니어링, RAG, 파인튜닝)을 선택하세요.

1. 회사의 최신 제품 정보를 기반으로 고객 질문에 답변하는 챗봇
2. 모든 마케팅 문구를 특정 브랜드 톤앤매너로 작성하는 봇
3. 간단한 텍스트 번역 작업

In [ ]:
# 정답
scenario_1 = "RAG"  # 최신 정보가 계속 변경되므로 RAG가 적합
scenario_2 = "Fine-tuning"  # 일관된 스타일/톤앤매너 유지는 파인튜닝
scenario_3 = "Prompting"  # 간단한 작업은 프롬프트로 충분

print("각 상황에 적합한 기술:")
print("=" * 60)
print(f"\n1. 최신 제품 정보 챗봇: {scenario_1}")
print("   이유: 제품 정보가 자주 업데이트되므로, 벡터 DB에 최신 정보를 저장하고")
print("         검색하여 답변하는 RAG가 적합합니다.")

print(f"\n2. 브랜드 톤앤매너 유지: {scenario_2}")
print("   이유: 모델의 '행동 방식'을 변경해야 하므로 파인튜닝이 적합합니다.")
print("         일관된 스타일과 형식을 유지하는 것이 목표입니다.")

print(f"\n3. 간단한 번역: {scenario_3}")
print("   이유: LLM은 이미 번역 능력이 있으므로, 적절한 프롬프트만으로 충분합니다.")
print("         추가 학습이나 외부 지식이 필요하지 않습니다.")

**풀이 설명**

- **접근 방법**: 각 기술의 강점과 적합한 상황을 매칭
- **핵심 개념**:
  - **RAG**: 최신 정보, 변경되는 지식, 사실 기반 답변이 필요할 때
  - **파인튜닝**: 일관된 스타일/톤/형식, 도메인 언어, 행동 변화가 필요할 때
  - **프롬프팅**: 간단한 작업, 빠른 프로토타입, LLM이 이미 할 수 있는 작업
- **실무 팁**: 복잡한 프로젝트에서는 세 기법을 조합하여 사용

---

### Q4. 학습 데이터 생성 (응용)

**문제**: "친근한 말투의 AI 비서"를 파인튜닝하기 위한 학습 데이터 3개를 JSONL 형식으로 작성하세요. 각 예시는 다른 주제(날씨, 일정, 추천)여야 합니다.

In [ ]:
# 정답
import json

# 시스템 메시지 정의
system_message = "당신은 친근하고 따뜻한 말투를 사용하는 AI 비서입니다. 반말 존댓말을 섞어 친구처럼 대화합니다."

# 학습 데이터 생성
training_data = [
    # 1. 날씨 관련
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "오늘 날씨 어때?"},
            {"role": "assistant", "content": "오늘 서울은 맑고 따뜻해요! 낮 최고 기온이 22도 정도 될 것 같은데, 가벼운 겉옷 하나 챙기면 딱 좋을 것 같아요. 혹시 외출 계획 있어요? 산책하기 좋은 날씨예요!"}
        ]
    },
    # 2. 일정 관련
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "내일 일정 뭐 있어?"},
            {"role": "assistant", "content": "내일 일정 확인해봤어요! 오전 10시에 팀 미팅이 있고, 오후 3시에 클라이언트 콜이 잡혀있네요. 저녁에는 특별한 약속 없어요. 혹시 추가하고 싶은 일정 있어요?"}
        ]
    },
    # 3. 추천 관련
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "점심 뭐 먹을까?"},
            {"role": "assistant", "content": "음, 오늘 점심 고민되죠? 날씨가 좋으니까 테라스 있는 파스타집 어때요? 아니면 든든하게 국밥도 좋고! 최근에 인기 있는 근처 맛집으로는 '봉주르 비스트로'가 있는데, 후기가 진짜 좋더라고요. 어떤 게 끌려요?"}
        ]
    }
]

# JSONL 형식으로 출력
print("친근한 AI 비서 학습 데이터 (JSONL 형식):")
print("=" * 60)

for i, data in enumerate(training_data, 1):
    jsonl_line = json.dumps(data, ensure_ascii=False)
    print(f"\n[{i}] {data['messages'][1]['content']}")
    print(f"    -> {data['messages'][2]['content'][:50]}...")

# 파일로 저장
import os
os.makedirs("datasets", exist_ok=True)

with open("datasets/friendly_assistant_data.jsonl", "w", encoding="utf-8") as f:
    for data in training_data:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

print("\n\nJSONL 파일 저장 완료: datasets/friendly_assistant_data.jsonl")

**풀이 설명**

- **접근 방법**: 일관된 말투 스타일을 유지하면서 다양한 주제의 예시 생성
- **핵심 개념**:
  - 시스템 메시지로 전체적인 톤앤매너 정의
  - 각 예시가 동일한 스타일(친근함)을 유지
  - 다양한 주제로 일반화 능력 향상
- **대안**: 더 많은 예시 (50개 이상) 추가하면 더 일관된 결과
- **실무 팁**: 실제 서비스에서 수집한 대화 데이터를 활용하면 더 자연스러운 학습 가능

---

### Q5. 파인튜닝 작업 관리 (응용)

**문제**: 다음 작업을 수행하는 코드를 작성하세요.

1. 현재 진행 중인 파인튜닝 작업 목록 확인
2. 가장 최근 작업의 상태와 이벤트 로그 출력

In [ ]:
# 정답

# 1. 파인튜닝 작업 목록 확인
print("파인튜닝 작업 목록:")
print("=" * 60)

jobs = client.fine_tuning.jobs.list(limit=10)

if jobs.data:
    for job in jobs.data:
        print(f"\n작업 ID: {job.id}")
        print(f"  상태: {job.status}")
        print(f"  모델: {job.model}")
        if job.fine_tuned_model:
            print(f"  결과 모델: {job.fine_tuned_model}")
else:
    print("  진행 중인 작업이 없습니다.")

In [ ]:
# 2. 가장 최근 작업의 상태와 이벤트 로그
print("\n가장 최근 작업 상세 정보:")
print("=" * 60)

if jobs.data:
    latest_job = jobs.data[0]
    
    print(f"\n작업 ID: {latest_job.id}")
    print(f"상태: {latest_job.status}")
    print(f"기반 모델: {latest_job.model}")
    print(f"생성 시간: {latest_job.created_at}")
    
    if latest_job.finished_at:
        print(f"완료 시간: {latest_job.finished_at}")
    
    if latest_job.fine_tuned_model:
        print(f"파인튜닝 모델: {latest_job.fine_tuned_model}")
    
    if latest_job.error:
        print(f"에러: {latest_job.error}")
    
    # 이벤트 로그
    print("\n이벤트 로그 (최근 5개):")
    print("-" * 40)
    
    events = client.fine_tuning.jobs.list_events(latest_job.id, limit=5)
    for event in events.data:
        print(f"  [{event.created_at}] {event.message}")
else:
    print("  파인튜닝 작업이 없습니다.")

**풀이 설명**

- **접근 방법**: OpenAI Fine-tuning API의 jobs.list()와 list_events() 활용
- **핵심 개념**:
  - `jobs.list()`: 파인튜닝 작업 목록 조회
  - `jobs.retrieve(job_id)`: 특정 작업 상세 조회
  - `jobs.list_events(job_id)`: 작업 이벤트 로그 조회
- **주의사항**: API 호출에 실패할 수 있으므로 예외 처리 권장
- **실무 팁**: 파인튜닝 작업은 수 분~수 시간 소요되므로 정기적 모니터링 필요

---

### Q6. 비용 계산 (응용)

**문제**: 다음 조건의 파인튜닝 비용을 계산하세요.

- 학습 데이터: 200개 예시
- 예시당 평균 토큰: 800
- 에포크: 5
- 모델: gpt-4o-mini ($3/1M 학습 토큰)

In [ ]:
# 정답

# 조건 설정
num_examples = 200
avg_tokens_per_example = 800
epochs = 5
cost_per_1m_tokens = 3.0  # gpt-4o-mini 학습 비용

# 계산
total_tokens = num_examples * avg_tokens_per_example * epochs
total_cost = (total_tokens / 1_000_000) * cost_per_1m_tokens

print("파인튜닝 비용 계산:")
print("=" * 50)
print(f"\n입력 조건:")
print(f"  - 학습 예시 수: {num_examples:,}개")
print(f"  - 예시당 평균 토큰: {avg_tokens_per_example:,}")
print(f"  - 에포크: {epochs}")
print(f"  - 모델: gpt-4o-mini")
print(f"  - 학습 비용: ${cost_per_1m_tokens}/1M 토큰")

print(f"\n계산 과정:")
print(f"  총 학습 토큰 = {num_examples} × {avg_tokens_per_example} × {epochs}")
print(f"              = {total_tokens:,} 토큰")
print(f"")
print(f"  비용 = ({total_tokens:,} / 1,000,000) × ${cost_per_1m_tokens}")
print(f"       = {total_tokens / 1_000_000:.3f} × ${cost_per_1m_tokens}")

print(f"\n예상 비용: ${total_cost:.2f}")

**풀이 설명**

- **접근 방법**: 총 토큰 수 계산 후 비용 산출
- **핵심 공식**: `총 비용 = (예시 수 × 토큰 수 × 에포크 / 1M) × 비용`
- **결과**: 200 × 800 × 5 = 800,000 토큰 → $2.40
- **실무 팁**: 실제 비용은 추론 비용도 포함해야 함. 파인튜닝된 모델의 추론 비용은 기본 모델보다 약간 높음

---

### Q7. 데이터 품질 검증 (복합)

**문제**: JSONL 파일을 읽어 다음을 검증하는 함수를 작성하세요.

1. 각 줄이 유효한 JSON인지 확인
2. 필수 키(messages)가 있는지 확인
3. 각 메시지에 role과 content가 있는지 확인
4. 총 예시 수와 평균 토큰 수 출력

In [ ]:
# 정답
import json
import tiktoken

def validate_jsonl(file_path):
    """JSONL 파일 품질 검증"""
    
    print(f"JSONL 파일 검증: {file_path}")
    print("=" * 60)
    
    errors = []
    valid_examples = []
    total_tokens = 0
    
    # tiktoken 인코더 (gpt-4o용)
    try:
        encoding = tiktoken.encoding_for_model("gpt-4o")
    except:
        encoding = tiktoken.get_encoding("cl100k_base")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if not line:  # 빈 줄 스킵
                    continue
                
                # 1. JSON 유효성 검사
                try:
                    data = json.loads(line)
                except json.JSONDecodeError as e:
                    errors.append(f"Line {line_num}: 유효하지 않은 JSON - {e}")
                    continue
                
                # 2. messages 키 확인
                if 'messages' not in data:
                    errors.append(f"Line {line_num}: 'messages' 키가 없습니다.")
                    continue
                
                # 3. 각 메시지의 role, content 확인
                messages = data['messages']
                line_valid = True
                line_tokens = 0
                
                for msg_idx, msg in enumerate(messages):
                    if 'role' not in msg:
                        errors.append(f"Line {line_num}, Message {msg_idx}: 'role' 키가 없습니다.")
                        line_valid = False
                    if 'content' not in msg:
                        errors.append(f"Line {line_num}, Message {msg_idx}: 'content' 키가 없습니다.")
                        line_valid = False
                    else:
                        # 토큰 수 계산
                        line_tokens += len(encoding.encode(msg['content']))
                
                if line_valid:
                    valid_examples.append(data)
                    total_tokens += line_tokens
    
    except FileNotFoundError:
        print(f"에러: 파일을 찾을 수 없습니다 - {file_path}")
        return None
    
    # 결과 출력
    print(f"\n검증 결과:")
    print(f"  - 총 예시 수: {len(valid_examples)}개")
    
    if valid_examples:
        avg_tokens = total_tokens / len(valid_examples)
        print(f"  - 총 토큰 수: {total_tokens:,}")
        print(f"  - 평균 토큰 수: {avg_tokens:.1f}")
    
    if errors:
        print(f"\n발견된 오류 ({len(errors)}개):")
        for error in errors[:5]:  # 처음 5개만 출력
            print(f"  - {error}")
        if len(errors) > 5:
            print(f"  ... 외 {len(errors) - 5}개")
    else:
        print(f"\n모든 검증 통과!")
    
    return {
        'valid_examples': len(valid_examples),
        'total_tokens': total_tokens,
        'avg_tokens': total_tokens / len(valid_examples) if valid_examples else 0,
        'errors': errors
    }

# 테스트
# result = validate_jsonl("datasets/news_finetuning_data.jsonl")
print("validate_jsonl() 함수 정의 완료")
print("\n사용법: validate_jsonl('파일경로.jsonl')")

**풀이 설명**

- **접근 방법**: 파일을 한 줄씩 읽으면서 각 조건을 순차적으로 검증
- **핵심 개념**:
  - JSON 파싱 오류 감지 (try-except)
  - 필수 키 존재 여부 확인
  - tiktoken으로 정확한 토큰 수 계산
- **주의사항**: 빈 줄 처리, 인코딩(UTF-8) 명시
- **실무 팁**: 실제 업로드 전 항상 데이터 검증 수행. OpenAI도 자체 검증을 하지만 사전 확인 권장

---

### Q8. 모델 비교 함수 (복합)

**문제**: 기본 모델과 파인튜닝된 모델의 응답을 비교하는 함수를 작성하세요.

요구사항:
1. 동일한 프롬프트로 두 모델 호출
2. 응답 내용, 응답 시간, 토큰 사용량 비교
3. 결과를 딕셔너리로 반환

In [ ]:
# 정답
import time

def compare_models(prompt, base_model, finetuned_model, system_message=None):
    """
    기본 모델과 파인튜닝된 모델의 응답을 비교
    
    Parameters:
    - prompt: 테스트할 프롬프트
    - base_model: 기본 모델 ID (예: "gpt-4o-mini")
    - finetuned_model: 파인튜닝된 모델 ID (예: "ft:gpt-4o-mini:...")
    - system_message: 시스템 메시지 (선택)
    
    Returns:
    - dict: 비교 결과
    """
    
    results = {
        'prompt': prompt,
        'base_model': {},
        'finetuned_model': {}
    }
    
    # 메시지 준비
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": prompt})
    
    # 1. 기본 모델 호출
    print(f"기본 모델 ({base_model}) 호출 중...")
    start_time = time.time()
    
    base_response = client.chat.completions.create(
        model=base_model,
        messages=messages,
        temperature=0.3
    )
    
    base_time = time.time() - start_time
    
    results['base_model'] = {
        'response': base_response.choices[0].message.content,
        'response_time': round(base_time, 2),
        'prompt_tokens': base_response.usage.prompt_tokens,
        'completion_tokens': base_response.usage.completion_tokens,
        'total_tokens': base_response.usage.total_tokens
    }
    
    # 2. 파인튜닝 모델 호출
    print(f"파인튜닝 모델 ({finetuned_model[:30]}...) 호출 중...")
    start_time = time.time()
    
    # 파인튜닝 모델은 시스템 메시지 없이도 학습된 스타일로 응답
    ft_messages = [{"role": "user", "content": prompt}]
    
    ft_response = client.chat.completions.create(
        model=finetuned_model,
        messages=ft_messages,
        temperature=0.3
    )
    
    ft_time = time.time() - start_time
    
    results['finetuned_model'] = {
        'response': ft_response.choices[0].message.content,
        'response_time': round(ft_time, 2),
        'prompt_tokens': ft_response.usage.prompt_tokens,
        'completion_tokens': ft_response.usage.completion_tokens,
        'total_tokens': ft_response.usage.total_tokens
    }
    
    # 3. 비교 결과 출력
    print("\n" + "=" * 70)
    print("모델 비교 결과")
    print("=" * 70)
    
    print(f"\n프롬프트: {prompt}")
    
    print(f"\n[기본 모델]")
    print(f"  응답: {results['base_model']['response'][:200]}...")
    print(f"  응답 시간: {results['base_model']['response_time']}초")
    print(f"  토큰: {results['base_model']['total_tokens']} (입력: {results['base_model']['prompt_tokens']}, 출력: {results['base_model']['completion_tokens']})")
    
    print(f"\n[파인튜닝 모델]")
    print(f"  응답: {results['finetuned_model']['response'][:200]}...")
    print(f"  응답 시간: {results['finetuned_model']['response_time']}초")
    print(f"  토큰: {results['finetuned_model']['total_tokens']} (입력: {results['finetuned_model']['prompt_tokens']}, 출력: {results['finetuned_model']['completion_tokens']})")
    
    # 토큰 절감율 계산
    if results['base_model']['prompt_tokens'] > 0:
        prompt_token_saving = (1 - results['finetuned_model']['prompt_tokens'] / results['base_model']['prompt_tokens']) * 100
        print(f"\n프롬프트 토큰 절감율: {prompt_token_saving:.1f}%")
    
    return results

print("compare_models() 함수 정의 완료")
print("\n사용법:")
print('result = compare_models("질문", "gpt-4o-mini", "ft:gpt-4o-mini:...")')

**풀이 설명**

- **접근 방법**: 두 모델을 동일한 조건에서 호출하고 결과 비교
- **핵심 개념**:
  - 응답 시간 측정 (time 모듈)
  - usage 객체에서 토큰 정보 추출
  - 파인튜닝 모델은 시스템 메시지 생략 가능 (토큰 절감)
- **주의사항**: 네트워크 상태에 따라 응답 시간이 달라질 수 있음
- **실무 팁**: 여러 번 테스트하여 평균값 비교 권장

---

### Q9. 도메인 특화 데이터셋 생성 (종합)

**문제**: 특정 도메인(예: 법률, 의료, 금융 중 선택)에 특화된 Q&A 데이터셋을 5개 이상 생성하고, JSONL 파일로 저장하세요.

요구사항:
1. 일관된 답변 형식 유지
2. 도메인 전문 용어 포함
3. 다양한 질문 유형

In [ ]:
# 정답: 금융 도메인 Q&A 데이터셋
import json
import os

# 시스템 메시지 정의
system_message = """당신은 금융 전문 상담사입니다. 
고객의 질문에 다음 형식으로 답변합니다:
1. [핵심 답변]: 질문에 대한 명확한 답변
2. [상세 설명]: 관련 금융 용어와 함께 자세한 설명
3. [주의사항]: 고객이 알아야 할 위험이나 고려사항"""

# 금융 Q&A 데이터셋
finance_qa_data = [
    {
        "question": "ETF와 펀드의 차이점이 뭔가요?",
        "answer": """1. [핵심 답변]: ETF(상장지수펀드)는 주식처럼 실시간 거래가 가능한 펀드이고, 일반 펀드는 하루 한 번 정해진 가격으로 거래됩니다.

2. [상세 설명]: ETF는 특정 지수(예: 코스피200)를 추종하며, 증권거래소에 상장되어 주식처럼 매수/매도가 가능합니다. 반면 일반 펀드는 펀드매니저가 종목을 선정하여 운용하며, 기준가격(NAV)으로 환매됩니다. ETF는 운용보수가 0.1~0.5%로 낮고, 펀드는 1~2% 수준입니다.

3. [주의사항]: ETF도 시장 위험에 노출되어 원금 손실 가능성이 있습니다. 레버리지 ETF는 변동성이 2~3배 크므로 단기 투자에만 적합합니다."""
    },
    {
        "question": "신용등급과 대출 금리는 어떤 관계가 있나요?",
        "answer": """1. [핵심 답변]: 신용등급이 높을수록 대출 금리가 낮아지고, 낮을수록 금리가 높아집니다. 1등급과 10등급 사이에는 연 5~10%p 이상의 금리 차이가 발생할 수 있습니다.

2. [상세 설명]: 신용등급은 1~10등급으로 나뉘며, NICE와 KCB 두 기관에서 평가합니다. 1~3등급은 '우량', 4~6등급은 '일반', 7~10등급은 '저신용'으로 분류됩니다. 은행 신용대출은 주로 1~6등급에게 제공되며, 7등급 이하는 저축은행이나 캐피탈 이용이 필요합니다.

3. [주의사항]: 단기간에 여러 곳에서 대출 조회를 하면 신용점수가 하락할 수 있습니다. 신용카드 연체, 휴대폰 요금 미납도 등급 하락 요인입니다."""
    },
    {
        "question": "적금과 예금 중 어떤 것이 유리한가요?",
        "answer": """1. [핵심 답변]: 목돈이 있다면 예금이, 매월 일정 금액을 저축할 계획이라면 적금이 적합합니다. 동일 금리 조건에서는 예금의 이자가 더 많습니다.

2. [상세 설명]: 정기예금은 목돈을 한 번에 예치하므로 전체 기간 동안 원금에 이자가 붙습니다. 적금은 매월 납입하므로 첫 달 납입금만 전체 기간 이자를 받습니다. 예를 들어 연 5% 금리로 1년간 1,200만원을 운용할 경우, 예금 이자는 약 60만원, 적금 이자는 약 32.5만원입니다.

3. [주의사항]: 중도 해지 시 약정 금리보다 낮은 금리가 적용됩니다. 예금자보호는 금융기관당 5,000만원까지만 보장됩니다."""
    },
    {
        "question": "DSR이 뭐고, 왜 중요한가요?",
        "answer": """1. [핵심 답변]: DSR(총부채원리금상환비율)은 연간 소득 대비 모든 대출의 원리금 상환액 비율입니다. 대출 한도를 결정하는 핵심 지표로, 현재 40~50% 규제가 적용됩니다.

2. [상세 설명]: DSR = (모든 대출의 연간 원리금 상환액 / 연간 소득) × 100으로 계산합니다. 예를 들어 연소득 6,000만원인 사람이 DSR 40% 규제를 받으면, 연간 원리금 상환액이 2,400만원을 초과할 수 없습니다. 주택담보대출, 신용대출, 학자금대출 등 모든 대출이 포함됩니다.

3. [주의사항]: 기존 대출이 있으면 신규 대출 한도가 줄어듭니다. 주택 구매 전 DSR 여유를 미리 확인하세요. 소득 증빙이 어려운 자영업자는 인정소득 기준이 까다롭습니다."""
    },
    {
        "question": "ISA 계좌의 장점은 무엇인가요?",
        "answer": """1. [핵심 답변]: ISA(개인종합자산관리계좌)는 다양한 금융상품을 한 계좌에서 운용하며, 수익에 대해 200~400만원까지 비과세, 초과분은 9.9% 분리과세 혜택을 받습니다.

2. [상세 설명]: ISA는 예금, 펀드, ETF, 리츠 등을 하나의 계좌에서 운용합니다. 3년 이상 유지 시 일반형은 200만원, 서민형/농어민형은 400만원까지 비과세입니다. 손익통산이 적용되어, 상품 A에서 100만원 손실, 상품 B에서 300만원 수익이면 순이익 200만원에만 과세합니다.

3. [주의사항]: 의무가입 기간(3년) 이전 해지 시 세제 혜택이 사라지고 일반 과세가 적용됩니다. 1인 1계좌만 가능하며, 금융소득종합과세 대상자는 가입이 제한됩니다."""
    },
    {
        "question": "주식 양도소득세는 어떻게 계산하나요?",
        "answer": """1. [핵심 답변]: 국내 주식은 대주주(10억원 이상 또는 지분 1% 이상)만 과세 대상이며, 해외 주식은 연간 수익 250만원 초과분에 22% 세율이 적용됩니다.

2. [상세 설명]: 대주주의 국내 주식 양도세는 과세표준 3억원 이하 22%, 3억원 초과 27.5%입니다. 해외 주식은 모든 투자자가 대상이며, 연간 양도차익에서 250만원 공제 후 22%(지방세 포함)가 과세됩니다. 예: 미국 주식에서 1,000만원 수익 시 (1,000만 - 250만) × 22% = 165만원 세금.

3. [주의사항]: 해외 주식 양도소득은 다음 해 5월에 확정신고해야 합니다. 손실과 이익은 같은 해 내에서만 상계되며, 이월공제는 불가합니다."""
    }
]

# JSONL 형식으로 변환
jsonl_data = []
for qa in finance_qa_data:
    jsonl_data.append({
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": qa["question"]},
            {"role": "assistant", "content": qa["answer"]}
        ]
    })

# 파일 저장
os.makedirs("datasets", exist_ok=True)
output_path = "datasets/finance_qa_finetuning.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for data in jsonl_data:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

print(f"금융 Q&A 데이터셋 생성 완료!")
print(f"파일 경로: {output_path}")
print(f"예시 수: {len(jsonl_data)}개")

print("\n데이터 미리보기:")
for i, qa in enumerate(finance_qa_data[:2], 1):
    print(f"\n[{i}] Q: {qa['question']}")
    print(f"    A: {qa['answer'][:100]}...")

**풀이 설명**

- **접근 방법**: 금융 도메인 선택, 일관된 답변 형식(핵심/상세/주의사항) 설계
- **핵심 개념**:
  - 시스템 메시지로 답변 형식 정의
  - 금융 전문 용어(DSR, ETF, NAV 등) 포함
  - 다양한 질문 유형 (비교, 정의, 계산 등)
- **대안**: 법률(계약서, 판례), 의료(증상, 처방) 등 다른 도메인도 가능
- **실무 팁**: 실제 고객 질문 데이터를 수집하여 더 자연스러운 데이터셋 구축

---

### Q10. 파인튜닝 파이프라인 구축 (종합)

**문제**: 파인튜닝 전체 파이프라인을 클래스로 구현하세요.

요구 기능:
1. `prepare_data()`: 데이터 준비 및 JSONL 생성
2. `upload_and_train()`: 파일 업로드 및 파인튜닝 시작
3. `monitor()`: 작업 상태 모니터링
4. `evaluate()`: 파인튜닝된 모델 평가

In [ ]:
# 정답
import json
import os
import time
from openai import OpenAI

class FineTuningPipeline:
    """OpenAI 파인튜닝 전체 파이프라인을 관리하는 클래스"""
    
    def __init__(self, model_name="gpt-4o-mini-2024-07-18"):
        """파이프라인 초기화"""
        self.client = OpenAI()
        self.model_name = model_name
        self.training_file_id = None
        self.job_id = None
        self.finetuned_model_id = None
        self.data_path = None
        
        print(f"FineTuningPipeline 초기화 완료")
        print(f"기본 모델: {self.model_name}")
    
    def prepare_data(self, data, output_path, system_message=None):
        """
        학습 데이터를 JSONL 형식으로 준비
        
        Parameters:
        - data: 학습 데이터 리스트 [{'input': '...', 'output': '...'}, ...]
        - output_path: JSONL 파일 저장 경로
        - system_message: 시스템 메시지 (선택)
        
        Returns:
        - str: 저장된 파일 경로
        """
        print("\n[Step 1] 데이터 준비 중...")
        
        # 디렉토리 생성
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # JSONL 형식으로 변환
        jsonl_data = []
        for item in data:
            messages = []
            
            # 시스템 메시지 추가 (선택)
            if system_message:
                messages.append({"role": "system", "content": system_message})
            
            # 사용자 입력과 이상적 출력
            messages.append({"role": "user", "content": item.get('input', item.get('question', ''))})
            messages.append({"role": "assistant", "content": item.get('output', item.get('answer', ''))})
            
            jsonl_data.append({"messages": messages})
        
        # 파일 저장
        with open(output_path, "w", encoding="utf-8") as f:
            for entry in jsonl_data:
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        
        self.data_path = output_path
        
        print(f"  - JSONL 파일 생성: {output_path}")
        print(f"  - 학습 예시 수: {len(jsonl_data)}개")
        
        return output_path
    
    def upload_and_train(self, file_path=None, hyperparameters=None):
        """
        파일 업로드 및 파인튜닝 작업 시작
        
        Parameters:
        - file_path: JSONL 파일 경로 (없으면 self.data_path 사용)
        - hyperparameters: 하이퍼파라미터 딕셔너리 (선택)
        
        Returns:
        - str: 파인튜닝 작업 ID
        """
        file_path = file_path or self.data_path
        if not file_path:
            raise ValueError("파일 경로가 지정되지 않았습니다.")
        
        print("\n[Step 2] 파일 업로드 및 파인튜닝 시작...")
        
        # 파일 업로드
        with open(file_path, "rb") as f:
            training_file = self.client.files.create(
                file=f,
                purpose="fine-tune"
            )
        
        self.training_file_id = training_file.id
        print(f"  - 파일 업로드 완료: {self.training_file_id}")
        
        # 파인튜닝 작업 생성
        job_params = {
            "training_file": self.training_file_id,
            "model": self.model_name
        }
        
        if hyperparameters:
            job_params["hyperparameters"] = hyperparameters
        
        fine_tuning_job = self.client.fine_tuning.jobs.create(**job_params)
        
        self.job_id = fine_tuning_job.id
        print(f"  - 파인튜닝 작업 생성: {self.job_id}")
        print(f"  - 상태: {fine_tuning_job.status}")
        
        return self.job_id
    
    def monitor(self, check_interval=60, max_wait_time=7200):
        """
        파인튜닝 작업 모니터링
        
        Parameters:
        - check_interval: 상태 확인 간격 (초)
        - max_wait_time: 최대 대기 시간 (초)
        
        Returns:
        - str: 파인튜닝된 모델 ID (성공 시) 또는 None (실패 시)
        """
        if not self.job_id:
            raise ValueError("파인튜닝 작업 ID가 없습니다. upload_and_train()을 먼저 실행하세요.")
        
        print(f"\n[Step 3] 파인튜닝 모니터링 (매 {check_interval}초 확인)...")
        
        start_time = time.time()
        
        while True:
            elapsed = time.time() - start_time
            if elapsed > max_wait_time:
                print(f"\n최대 대기 시간 초과 ({max_wait_time}초)")
                return None
            
            job_info = self.client.fine_tuning.jobs.retrieve(self.job_id)
            status = job_info.status
            
            print(f"  [{time.strftime('%H:%M:%S')}] 상태: {status}")
            
            if status == "succeeded":
                self.finetuned_model_id = job_info.fine_tuned_model
                print(f"\n  파인튜닝 완료!")
                print(f"  모델 ID: {self.finetuned_model_id}")
                return self.finetuned_model_id
            
            elif status == "failed":
                print(f"\n  파인튜닝 실패: {job_info.error}")
                return None
            
            elif status == "cancelled":
                print(f"\n  파인튜닝 취소됨")
                return None
            
            time.sleep(check_interval)
    
    def evaluate(self, test_prompts, system_message=None):
        """
        파인튜닝된 모델 평가
        
        Parameters:
        - test_prompts: 테스트할 프롬프트 리스트
        - system_message: 기본 모델용 시스템 메시지
        
        Returns:
        - list: 평가 결과 리스트
        """
        if not self.finetuned_model_id:
            raise ValueError("파인튜닝된 모델 ID가 없습니다.")
        
        print(f"\n[Step 4] 모델 평가...")
        print(f"  테스트 프롬프트: {len(test_prompts)}개")
        
        results = []
        
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\n  [{i}/{len(test_prompts)}] 테스트 중...")
            
            # 기본 모델 응답
            base_messages = []
            if system_message:
                base_messages.append({"role": "system", "content": system_message})
            base_messages.append({"role": "user", "content": prompt})
            
            base_response = self.client.chat.completions.create(
                model=self.model_name,
                messages=base_messages,
                temperature=0.3
            )
            
            # 파인튜닝 모델 응답
            ft_response = self.client.chat.completions.create(
                model=self.finetuned_model_id,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3
            )
            
            results.append({
                'prompt': prompt,
                'base_response': base_response.choices[0].message.content,
                'ft_response': ft_response.choices[0].message.content,
                'base_tokens': base_response.usage.total_tokens,
                'ft_tokens': ft_response.usage.total_tokens
            })
        
        # 결과 요약
        print("\n" + "=" * 60)
        print("평가 결과 요약")
        print("=" * 60)
        
        for r in results:
            print(f"\nQ: {r['prompt'][:50]}...")
            print(f"  기본 모델 ({r['base_tokens']} tokens): {r['base_response'][:100]}...")
            print(f"  파인튜닝 ({r['ft_tokens']} tokens): {r['ft_response'][:100]}...")
        
        return results
    
    def get_status(self):
        """현재 파이프라인 상태 확인"""
        print("\n파이프라인 상태:")
        print(f"  - 기본 모델: {self.model_name}")
        print(f"  - 데이터 경로: {self.data_path or '미설정'}")
        print(f"  - 학습 파일 ID: {self.training_file_id or '미설정'}")
        print(f"  - 작업 ID: {self.job_id or '미설정'}")
        print(f"  - 파인튜닝 모델: {self.finetuned_model_id or '미설정'}")


# 사용 예시
print("FineTuningPipeline 클래스 정의 완료")
print("\n사용 예시:")
print("""pipeline = FineTuningPipeline()
pipeline.prepare_data(training_data, "datasets/my_data.jsonl")
pipeline.upload_and_train()
pipeline.monitor(check_interval=60)
pipeline.evaluate(["테스트 질문1", "테스트 질문2"])""")

**풀이 설명**

- **접근 방법**: 파인튜닝 전체 과정을 단계별 메서드로 분리하여 클래스로 캡슐화
- **핵심 개념**:
  - `prepare_data()`: 데이터 형식 변환 및 파일 생성
  - `upload_and_train()`: OpenAI API를 통한 파일 업로드 및 작업 생성
  - `monitor()`: 폴링 방식의 상태 모니터링
  - `evaluate()`: 기본 모델과 파인튜닝 모델 비교
- **확장 방향**: 검증 데이터 분리, 로깅, 에러 핸들링, 비용 추정 등 추가 가능
- **실무 팁**: 프로덕션에서는 클라우드 스토리지와 연동하여 대용량 데이터 처리

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| 파인튜닝 | 모델의 내부 가중치를 업데이트 | 스타일/톤/형식 일관성 |
| Prompting vs RAG vs FT | 각각 다른 문제 해결 | 상황에 맞는 기술 선택 |
| JSONL 형식 | messages 배열 (system, user, assistant) | 학습 데이터 준비 |
| 데이터 품질 | 일관성, 다양성, 정확성 | 최소 50개 이상 권장 |
| 파일 업로드 | client.files.create() | purpose="fine-tune" |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 기법 | 권장 설정 |
|------|----------|----------|
| 파인튜닝 API | client.fine_tuning.jobs.create() | gpt-4o-mini 권장 |
| 모니터링 | jobs.retrieve(), list_events() | 60초 간격 체크 |
| 하이퍼파라미터 | n_epochs, learning_rate_multiplier | 자동 설정 우선 |
| 비용 최적화 | 작은 모델, 품질 데이터, 짧은 프롬프트 | $3/1M 토큰 (학습) |

### 파인튜닝 체크리스트

```
1. 사전 준비:
   [x] 파인튜닝이 적합한 상황인지 확인 (스타일/톤/형식 변경)
   [x] 고품질 예시 데이터 50개 이상 준비
   [x] 일관된 형식/스타일 유지
   
2. 데이터 준비:
   [x] JSONL 형식으로 변환 (ensure_ascii=False)
   [x] messages 구조 검증 (system, user, assistant)
   [x] 토큰 수 확인 (너무 길지 않게)
   
3. 파인튜닝 실행:
   [x] 파일 업로드 (purpose="fine-tune")
   [x] 작업 생성 (모델, 하이퍼파라미터)
   [x] 상태 모니터링 (jobs.retrieve)
   
4. 평가 및 배포:
   [x] 기본 모델과 성능 비교
   [x] 다양한 테스트 케이스 검증
   [x] 프로덕션 적용
```

### 실무 팁

1. **RAG vs 파인튜닝**: 지식 추가는 RAG, 행동 변화는 파인튜닝
2. **데이터 품질이 핵심**: 50개 고품질 > 500개 저품질
3. **작은 모델부터**: gpt-4o-mini로 시작, 필요시 gpt-4o로 업그레이드
4. **시스템 메시지 생략**: 파인튜닝 후에는 짧은 프롬프트로 충분
5. **비용 모니터링**: 학습 비용 + 추론 비용 함께 고려